In [ ]:
import healpy as hp
from pysm3 import units as u
import pysm3
import numpy as np
import matplotlib.pyplot as plt
import toml
from pathlib import Path

In [ ]:
from astropy.table import QTable

In [ ]:
cd ..

In [ ]:
config = toml.load("common.toml")

In [ ]:
# chs = chs[chs["telescope"]=="SAT"]

In [ ]:
from astropy.io import fits

In [ ]:
gal2eq = hp.Rotator(coord="GC")

In [ ]:
lines = [
           ( "10", 115.271 * u.GHz),
           ( "21", 230.538 * u.GHz),
            ("32", 345.796 * u.GHz),
]

In [ ]:
co_line_230_input ={lines[i][0] : (gal2eq.rotate_map_alms(
            hp.smoothing(
            hp.ud_grade(hp.read_map("/mnt/home/azonca/s/pysm-data/co/HFI_CompMap_CO-Type1_2048_R2.00_ring.fits", i), 512), fwhm=np.radians(1))) * u.K_CMB).to(u.uK_CMB) for i in range(3)}

In [ ]:
hp.mollview(co_line_230_input["21"], min=0, max=1000, title="Input 230 GHz CO line map from Planck")

In [ ]:
import mapsims

In [ ]:
chs = mapsims.parse_channels(instrument_parameters=config["instrument_parameters"])

In [ ]:
for ch in chs:
    for model in ["co1", "co3"]:
        print(ch, model, end=" - ")
        tag = "co_" + model
        folder = Path(config["output_folder"].format(tag=tag))
        filename = config["output_filename_template"].format(tag=tag, telescope=ch.telescope, nside=2048, band=ch.band)
        m = pysm3.read_map(folder / filename, nside=512, field=(0,1,2))

        # hp.mollview(m[0].value, min=0, max=10, unit=m.unit, title="CO low complexity")

        bandpass_frequency, bandpass_weight = ch.bandpass

        has_co = False
        for line_tag, line_freq in lines:
            if line_freq > bandpass_frequency[0] and line_freq < bandpass_frequency[-1]:
                has_co = True
                break
                
        if not has_co:
            if u.allclose(m,0 * u.uK_CMB):
                print("No CO emission")
            else:
                print("Error: there should be no CO emission, instead std of the map is", np.std(m[0]))
        else:
            # plt.axvline(0.022, color="black")
            # plt.hist(m[0]/co_line_230_input, 1000, range=[0,.1]);

            median_map_scale = np.median(m[0]/co_line_230_input[line_tag])

            mean_band_normalization = np.interp(line_freq, bandpass_frequency, pysm3.normalize_weights(bandpass_frequency.value, bandpass_weight))

            ratio = median_map_scale/mean_band_normalization - 1
            
            print(f"CO {line_tag} Emission at {line_freq} is {median_map_scale:.2g} compared to the Planck map, given the bandpass we expect roughly {mean_band_normalization:.2g}, the error is {ratio:.1%}")
            
            if model == "co1":
                assert u.allclose(m[1:], 0*u.uK_CMB), "CO1 should be unpolarized"
            else:
                median_map_scale_pol = np.median(np.sqrt(m[1]**2+m[2]**2)/m[0])
                print(f"In polarization (expected less than .1% due to depolarization): {median_map_scale_pol:.2%}")
                assert median_map_scale_pol < .1/100 and median_map_scale_pol > .01/100
            assert ratio > -.5 and ratio < .5